In [149]:
import numpy as np
import plotly.express as ex
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [135]:
class Hopfield:
    
    def __init__(self, bias):
        self.weights = None
        self.positions_to_update = []
        self.bias = bias

    # Compute the weights matrix
    def train(self, patterns, activity=.1):
        self.avg_activity = activity

        # Matrix multiply pattern as two column vectors, then sum over all patterns
        self.weights = np.sum([(p.reshape((-1,1))-self.avg_activity)@(p.reshape((1,-1))-self.avg_activity) for p in patterns], axis=0)
        
        """
        self.weights = np.zeros(((patterns[0].reshape((-1,1))@patterns[0].reshape((1,-1))).shape))
        for p in patterns: 
            self.weights += p.reshape((-1,1))@p.reshape((1,-1))
        """
        
        # The lab seems to want these to stay non-zero
        #np.fill_diagonal(self.weights, 0)
        
        self.weights = self.weights.astype(np.float64).copy()
        self.weights /= self.weights.shape[0]
        
        
    
    def calc_activity(self, patterns):
        """
        Calculates the average activity 
        """
        activated_patterns = self.update(patterns)
        self.avg_activity = 1 / (self.weights.shape[0] * patterns.shape[0]) * np.sum(activated_patterns)
    # Check if patterns are sucessfully memorised    
    def check(self, patterns, synchronous=True, verbose=True):        
        successes = 0
        for pattern in patterns:
            recalled = self.recall(pattern, synchronous=synchronous)
            
            if np.array_equal(recalled, pattern):
                successes += 1                
            else:
                if (verbose):
                    print("Warning: input pattern is not a fixed point ", pattern, recalled)

        if (verbose):
            print(successes, " patterns memorized successfully")
        return successes
            
        
    

    def update(self, pattern, synchronous=True):       
        if synchronous:
            pattern_update = np.sign(self.weights@pattern - self.bias)
            pattern_update[pattern_update==0] = 1
            pattern_update = 0.5 + 0.5 * pattern_update
           
        else:
            # List all patterns dimensions not updated yet (in this complete recall)
            if not self.positions_to_update:
                self.positions_to_update = [i for i in range(pattern.shape[0])]
            
            # Select randomly one position to update
            np.random.seed(2020)
            j = np.random.choice(self.positions_to_update)
            
            # Update the chosen position
            pattern_update = pattern.copy()
            value = np.sign(np.sum(self.weights[j,:]*pattern))
            pattern_update[j] = 1 if value>=0 else -1
            self.positions_to_update.remove(j)
        
        return pattern_update
    
    
    
    def recall(self, pattern, synchronous=True, max_iterations=1):
        iteration = 0
        while(iteration < max_iterations):
            if synchronous:
                pattern_update = self.update(pattern, synchronous=True)
            else:
                pattern_update = pattern
                for _ in range(pattern.shape[0]):
                    pattern_update = self.update(pattern_update, synchronous=False)
                    
            if np.array_equal(pattern_update, pattern):
                break
            else:
                iteration += 1
                pattern = pattern_update
    
        return pattern
        
   
    # Define a function to find all possible attractors
    def attractors(self, synchronous=True):
        attractors = set()
        all_possible_patterns = binary_patterns(self.weights.shape[1])
        for pattern in all_possible_patterns:
            attractors.add(tuple(self.recall(pattern, synchronous=synchronous, max_iterations=100)))
        
        return np.array([np.array(attractor) for attractor in attractors])
    
    
    def energy(self, state):
        alpha = state.reshape((-1,1))@state.reshape((1,-1))
        #return -np.sum(np.sum(alpha*self.weights,  axis=1), axis=0)
        return -np.sum(alpha*self.weights)
    

In [163]:
np.random.seed(2020)
size = 100
n = 25
bias=0
activity = 0.1

active_inputs = int(activity*size*n)
Training_patterns = np.zeros(n*size, dtype=np.int8)
idx = np.random.choice(len(Training_patterns), size=active_inputs, replace=False)
Training_patterns[idx] = 1
Training_patterns = np.hsplit(Training_patterns, n)


model = Hopfield(bias=0.04)
model.train(Training_patterns, 0.1)
model.check(Training_patterns, verbose=False)


23

In [213]:
np.random.seed(2020)
size = 100
n_samples = np.arange(1,25)
biases=np.linspace(-0.1, 0.1, 20)

activity = 0.1
suc_bias = []
for b in biases:
    suc_samples = []
    for n in n_samples:
        active_inputs = int(activity*size*n)
        Training_patterns = np.zeros(n*size, dtype=np.int8)
        idx = np.random.choice(len(Training_patterns), size=active_inputs, replace=False)
        Training_patterns[idx] = 1
        Training_patterns = np.hsplit(Training_patterns, n)

        model = Hopfield(bias=b)
        model.train(Training_patterns, activity)
        successes = model.check(Training_patterns, verbose=False)
        suc_samples.append(successes)

    suc_bias.append(np.asarray(suc_samples))
suc_bias = np.asarray(suc_bias)



In [214]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("% of memorized samples by N","% of memorized samples by bias"))

# col 1
for i in range(suc_bias.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(1,25), y=suc_bias[i]/np.arange(1,25), name="b={:.2f}".format(biases[i])), row=1, col=1)

#col 2
for i in range(1,24):
    fig.add_trace(go.Scatter(x=biases, y=suc_bias.T[i-1]/i, name="b={:.2f}".format(i)), row=1, col=2)

fig.show()

In [216]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("% of memorized samples by N","% of memorized samples by bias"))

# col 1
for i in range(0, suc_bias.shape[0], 4):
    fig.add_trace(go.Scatter(x=np.arange(1,25), y=suc_bias[i]/np.arange(1,25), name="b={:.2f}".format(biases[i])), row=1, col=1)

#col 2
for i in range(1,24, 5):
    fig.add_trace(go.Scatter(x=biases, y=suc_bias.T[i-1]/i, name="n={:.2f}".format(i)), row=1, col=2)

fig.show()

In [217]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("% of memorized samples by N","% of memorized samples by bias"))

# col 1
for i in range(10, suc_bias.shape[0], 4):
    fig.add_trace(go.Scatter(x=np.arange(1,25), y=suc_bias[i-1]/np.arange(1,25), name="b={:.2f}".format(biases[i-1])), row=1, col=1)

#col 2
for i in range(4,24, 8):
    fig.add_trace(go.Scatter(x=biases, y=suc_bias.T[i-1]/i, name="n={:.2f}".format(i)), row=1, col=2)

fig.show()

# Even sparser patterns

In [218]:
# p = 0.05

np.random.seed(2020)
size = 100
n_samples = np.arange(1,25)
biases=np.linspace(-0.1, 0.1, 20)

activity = 0.05

suc_bias = []
for b in biases:
    suc_samples = []
    for n in n_samples:
        active_inputs = int(activity*size*n)
        Training_patterns = np.zeros(n*size, dtype=np.int8)
        idx = np.random.choice(len(Training_patterns), size=active_inputs, replace=False)
        Training_patterns[idx] = 1
        Training_patterns = np.hsplit(Training_patterns, n)

        model = Hopfield(bias=b)
        model.train(Training_patterns, activity)
        successes = model.check(Training_patterns, verbose=False)
        suc_samples.append(successes)

    suc_bias.append(np.asarray(suc_samples))
suc_bias = np.asarray(suc_bias)



In [219]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("% of memorized samples by N","% of memorized samples by bias"))

# col 1
for i in range(suc_bias.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(1,25), y=suc_bias[i]/np.arange(1,25), name="b={:.2f}".format(biases[i])), row=1, col=1)

#col 2
for i in range(1,24):
    fig.add_trace(go.Scatter(x=biases, y=suc_bias.T[i-1]/i, name="b={:.2f}".format(i)), row=1, col=2)

fig.show()

In [224]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("% of memorized samples by N","% of memorized samples by bias"))

# col 1
for i in range(8, suc_bias.shape[0], 4):
    fig.add_trace(go.Scatter(x=np.arange(1,25), y=suc_bias[i-1]/np.arange(1,25), name="b={:.2f}".format(biases[i-1])), row=1, col=1)

#col 2
for i in range(4,24, 8):
    fig.add_trace(go.Scatter(x=biases, y=suc_bias.T[i-1]/i, name="n={:.2f}".format(i)), row=1, col=2)

fig.show()

In [227]:
# p = 0.01

np.random.seed(2020)
size = 100
n_samples = np.arange(1,25)
biases=np.linspace(-0.1, 0.1, 20)

activity = 0.01

suc_bias = []
for b in biases:
    suc_samples = []
    for n in n_samples:
        active_inputs = int(activity*size*n)
        Training_patterns = np.zeros(n*size, dtype=np.int8)
        idx = np.random.choice(len(Training_patterns), size=active_inputs, replace=False)
        Training_patterns[idx] = 1
        Training_patterns = np.hsplit(Training_patterns, n)

        model = Hopfield(bias=b)
        model.train(Training_patterns, activity)
        successes = model.check(Training_patterns, verbose=False)
        suc_samples.append(successes)

    suc_bias.append(np.asarray(suc_samples))
suc_bias = np.asarray(suc_bias)



In [228]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("% of memorized samples by N","% of memorized samples by bias"))

# col 1
for i in range(suc_bias.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(1,25), y=suc_bias[i]/np.arange(1,25), name="b={:.2f}".format(biases[i])), row=1, col=1)

#col 2
for i in range(1,24):
    fig.add_trace(go.Scatter(x=biases, y=suc_bias.T[i-1]/i, name="b={:.2f}".format(i)), row=1, col=2)

fig.show()

In [229]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("% of memorized samples by N","% of memorized samples by bias"))

# col 1
for i in range(8, suc_bias.shape[0], 4):
    fig.add_trace(go.Scatter(x=np.arange(1,25), y=suc_bias[i-1]/np.arange(1,25), name="b={:.2f}".format(biases[i-1])), row=1, col=1)

#col 2
for i in range(4,24, 8):
    fig.add_trace(go.Scatter(x=biases, y=suc_bias.T[i-1]/i, name="n={:.2f}".format(i)), row=1, col=2)

fig.show()